In [1]:
import os
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import splitfolders
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import mixed_precision
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation, Input, Average
from tensorflow.keras.applications import DenseNet121, ResNet50, EfficientNetB0
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
splitfolders.ratio(r"C:\Users\A D SARNITHA\OneDrive\Documents\Project\FYP phase1\Review 1\Diabeties-Detection-CNN-main\dataset", output="Output",
    seed=1337, ratio=(.75, .2, .05), group_prefix=None, move=False)


Copying files: 2899 files [00:08, 345.43 files/s]


In [3]:
# Directories
data_dir = pathlib.Path('Output/')
train_dir = 'Output/train'
val_dir = 'Output/val'
test_dir = 'Output/test'


In [4]:
# Data Generators
train_datagen = ImageDataGenerator(rescale=1/255.0, rotation_range=0.2, zoom_range=0.2, width_shift_range=0.2, 
                                   height_shift_range=0.2, vertical_flip=True, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1/255.0)
test_datagen = ImageDataGenerator(rescale=1/255.0)

train_data = train_datagen.flow_from_directory(train_dir, batch_size=32, target_size=(224, 224), class_mode='categorical', shuffle=True)
valid_data = valid_datagen.flow_from_directory(val_dir, batch_size=32, target_size=(224, 224), class_mode='categorical', shuffle=False)
test_data = test_datagen.flow_from_directory(test_dir, batch_size=32, target_size=(224, 224), class_mode='categorical', shuffle=False)


Found 2173 images belonging to 5 classes.
Found 578 images belonging to 5 classes.
Found 148 images belonging to 5 classes.


In [5]:
# Load Base Models
base_model1 = DenseNet121(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model2 = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model3 = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

for model in [base_model1, base_model2, base_model3]:
    model.trainable = False


In [6]:
# Model Inputs
inputs = Input(shape=(224, 224, 3))

In [7]:
# Feature Extraction
x1 = base_model1(inputs, training=False)
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(1024, activation='relu')(x1)  # Align to 1024

x2 = base_model2(inputs, training=False)
x2 = GlobalAveragePooling2D()(x2)
x2 = Dense(1024, activation='relu')(x2)  # Align to 1024

x3 = base_model3(inputs, training=False)
x3 = GlobalAveragePooling2D()(x3)
x3 = Dense(1024, activation='relu')(x3)  # Align to 1024

# Merge Outputs
merged = Average()([x1, x2, x3])
dropout = Dropout(0.25)(merged)
predictions = Dense(5, activation='softmax')(dropout)


In [8]:
mixed_precision.set_global_policy('mixed_float16')


In [ ]:
# Define Model
ensemble_model = Model(inputs, predictions)
ensemble_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
history = ensemble_model.fit(train_data, epochs=10, validation_data=valid_data)


C:\Users\A D SARNITHA\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
11/68 ━━━━━━━━━━━━━━━━━━━━ 4:30 5s/step - accuracy: 0.1915 - loss: 2.2831

In [ ]:
# Evaluate on Test Data
test_pred = ensemble_model.predict(test_data)
test_pred_labels = np.argmax(test_pred, axis=1)

# Print Classification Report
true_labels = test_data.classes
print(classification_report(true_labels, test_pred_labels))


In [ ]:
# Confusion Matrix
cfm_test = confusion_matrix(true_labels, test_pred_labels)
sns.heatmap(cfm_test, annot=True, cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Save Model
ensemble_model.save('ensemble_model.keras')


In [ ]:
# Load and Predict on a New Image
def predict_image(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = np.array(img) / 255.0
    img_array = img_array.reshape(1, 224, 224, 3)
    
    label = model.predict(img_array)
    class_names = ['Mild', 'Moderate', 'No_DR', 'Proliferate_DR', 'Severe']
    predicted_class_index = np.argmax(label)
    predicted_class = class_names[predicted_class_index]
    
    plt.imshow(img)
    plt.title(predicted_class)
    plt.axis('off')
    plt.show()
    
    return predicted_class

In [ ]:
# Example Usage
model = load_model('ensemble_model.keras')
image_path = r"C:\Users\A D SARNITHA\OneDrive\Documents\Project\FYP phase1\Review 1\Diabeties-Detection-CNN-main\output\test\Severe\c0a0828e01b4.png"  # Replace with actual image path
predicted_class = predict_image(image_path, model)
print("Predicted Class:", predicted_class)
